# Movie Review Analysis Outline
## import necessary python libraries


In [1]:
# output.txt is created by running a python script to convert the format of the movie data
# into a format that spark readily converts into a RDD

# "transformData.py" is this script which is intended to be run once on our data
import re
import string
from operator import add
import os
import sys

In [2]:
# Just notes for Lise... ignore if you arent having any spark not found problems
# Tried Configure the necessary Spark environment
# TOOK FOREVER TO CONFIGURE THIS.... pyspark... grrr
# dont try unless you know vaguely what variable this takes care of 
# export SPARK_HOME="/usr/local/bin/spark-1.3.1-bin-hadoop2.6"
"""
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")


# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, '/python/lib/py4j-0.8.2.1-src.zip'))

"""

from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint

import pyspark.mllib.regression as mllib_reg
import pyspark.mllib.linalg as mllib_lalg
import pyspark.mllib.classification as mllib_class
import pyspark.mllib.tree as mllib_tree

print (pyspark) # test to see that pyspark is up and running okay



<module 'pyspark' from '/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/__init__.py'>


# CLEAN DATA

In [3]:
def strip_html_tags(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data.lower())

#parse and take care of funky symbols in summary and text <- normalize to root words maybe?
def cleanData (data):
    data = strip_html_tags(data)
    data =  re.sub("[\t\,\:;\(\)\"\'\~\-\!\?\`]", "",data, 0, 0)
    data =  re.sub("[\.0`]", "",data, 0, 0) # special case to get rid of ".0" of scores -- 
    return data

#re.sub("[\.\t\,\:;\(\)\.]", " ", strip_html_tags(data.lower()), 0, 0)

# clean the data for each element of each sub list
def prepData (list_str):
    L= []
    for x in list_str:
        #print x
        L.append(cleanData(x))
    return L

# [u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text']
# headers for the project





# PARSE THE FILE WE WANT TO USE AS OUR DATA

In [4]:
# CHANGED SEPARATOR TO THREE /// because / separator was causing discrepencies
# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky

# output < smaller_parsed
#movies_txt = sc.textFile("output.txt").map(lambda x: (cleanData(x).split('///')))
movies_txt = sc.textFile("smaller_parsed.txt").map(lambda x: (cleanData(x).split('///')))

## Parsing out relevant fields - score and text of review

In [5]:
# prep our data


movies_new = movies_txt.map(lambda L: (L[0], L[4], L[6], L[7]) if len(L) == 8 else ("ERROR"))
movies_new1 = movies_new.filter(lambda L: L!=("ERROR"))
# check that we aren't losing any data 
# --> we are still missing some from our parsing.... :/ 
# 9 in fact... we can fix this later
print (movies_new.count())
print (movies_new1.count())
print (movies_new.count() == movies_new1.count())

removeHTMLTags = movies_new1.map(prepData) # remove html tags

111110
111101
False


## REVISED STOP WORDS REMOVAL 

In [6]:
#REVISED STOP WORDS REMOVAL Approach

#used the stopwords file in the virtual box instead... taking too long
baseDir = os.path.join('../data')
inputPath = os.path.join('cs100', 'lab3')
STOPWORDS_PATH = 'stopwords.txt'
split_regex = r'\W+'

stopfile = os.path.join(baseDir, inputPath, STOPWORDS_PATH)
stopwords = set(sc.textFile(stopfile).collect())
#print 'These are the stopwords: %s' % stopwords

def tokenize(string):
    #recycling assignement code
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    splitList = re.split(split_regex, string.lower().strip())
    
    return [x for x in splitList if x!="" and x not in stopwords]  


removeSW_output= removeHTMLTags.map(lambda x: (  int((str(x[1])).replace("/","")), tokenize(x[3]))   ).cache()
print(removeSW_output.top(2))
print (removeSW_output.count())

[(5, [u'zulu', u'superlative', u'depiction', u'19th', u'century', u'battle', u'rourkes', u'drift', u'british', u'outpost', u'heavily', u'outnumbered', u'seige', u'zulu', u'warriors', u'fought', u'desperately', u'almost', u'last', u'man', u'somehow', u'prevailed', u'terrifying', u'exciting', u'war', u'film', u'unlike', u'splendid', u'casta', u'word', u'diamond', u'entertainment', u'tapesmy', u'experience', u'slp', u'speed', u'vhs', u'videos', u'tracks', u'remarkably', u'well', u'usually', u'recorded', u'hifi', u'sound', u'well', u'standard', u'linear', u'formatzulu', u'also', u'available', u'dvdalso', u'recommendedzulu', u'dawn', u'1979', u'tells', u'complete', u'story', u'struggles', u'tribesmen', u'british', u'soldiers', u'commenced', u'1879', u'burt', u'lancaster', u'vhs', u'edition', u'dvd', u'editionparenthetical', u'number', u'preceding', u'title', u'1', u'1', u'viewer', u'poll', u'rating', u'found', u'film', u'resource', u'website8', u'zulu', u'uk1964', u'stanley', u'baker', u'ja

## Vectorizing - making the vector of features...

In [7]:
## Took forver to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams

htf = HashingTF(numFeatures=1) # features need to match dimensions of output

print "just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']"
print (htf.transform(["hi", "boo", "hi"]));
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))

#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
print feature_vector.filter(lambda x: x.label == 1.0).count()

(1,[0],[3.0])
95309


In [139]:
# EXPLORE DIFFERENT ML APPROACHS
# https://spark.apache.org/docs/1.1.0/api/python/pyspark.mllib.classification-module.html

##  Trying SVM approach

In [10]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint


'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - update number of features
'''
## Took forever to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams
htf = HashingTF(numFeatures=1) # features need to match dimensions of output
print ("SVM: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
'''
REFRESHED DATA
'''


training_data, testing_data= feature_vector.randomSplit([0.7, 0.3])



# Build the Support Vector Machine model
model = SVMWithSGD.train(feature_vector, iterations=50) # lower iterations so we arent here forever

# Evaluate the model on training data
labelsAndPreds = feature_vector.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(feature_vector.count())
print("training error of the model= " + str(trainErr))

#Evaluate on testing data.. the correct way lolz
###### TO DO 
#labelsAndPreds = feature_vector.map(lambda p: (p.label, model.predict(p.features)))
#trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(feature_vector.count())
#print("training error of the model= " + str(trainErr))

# Saving the model is meh
#model.save(sc, "target/tmp/pythonSVMWithSGDModel")
#sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

SVM: REFRESHING HTF, and FEATURE VECTOR
training error of the model= 0.857859065175
training error of the model= 0.857859065175


## Do some Predictions - Bayes

In [12]:
### THIS USES NAIVE BAYES CLASSIFIER
### with 116 elements with the first output.txt 86KB , gets 80-93%
# tested with lots of data.... - 111MB

'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - update number of features
'''
## Took forever to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams
htf = HashingTF(numFeatures=1) # features need to match dimensions of output
print ("BAYES: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
'''
REFRESHED DATA
'''

# would be nice to figure out which division is the best
training_data, testing_data= feature_vector.randomSplit([0.6, 0.4])
#training_data, testing_data= feature_vector.randomSplit([0.7, 0.3])

# parameters:
lamda = 1.0

# build Naive Bayes Classifier
nbay = mllib_class.NaiveBayes.train(training_data, lamda)

# Make prediction and test accuracy.
predictionAndLabel = testing_data.map(lambda p : (nbay.predict(p.features), p.label))
testErr = predictionAndLabel.filter(lambda (v, p): v != p).count() / float(testing_data.count())
accuracy = 100.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / testing_data.count()

print accuracy

BAYES: REFRESHING HTF, and FEATURE VECTOR
85.847718935
